# QUBO formulation of polynomial equation

# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
 1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 
$$

with $q$ and $p$ paramerers taking disrete values either 1 or 2

In [13]:
import numpy as np

def nlfunc(input):
    x0,x1,x2,x3 = input
    q, p = parameters

    def f0():
        return 0.5 - x0 + x1
    
    def f1():
        return 1 - x1
    
    def f2():
        return 2  - q*x0**2  - x2

    def f3():
        return -p*x1**2 + x2 - x3
    
    return np.array([f0(), f1(), f2(), f3()])



## 2. QUBO formalism for linear systems

The Quandratic Unconstrainted Binary Optimization problem, or QUBO, allows to minimize the cost function :

$$
E(x) = x^{T}Qx
$$

where the variables $x_i$ are binaries, i.e. the are 0 or 1. The equation above can be rewritten as :

$$
E(x) = \sum_i Q_{ii}x_i + \sum_{ij} Q_{ij}x_ix_j
$$

that is very similar to the Ising model, basis of the quantum annealler architecture. 

### Encoding real numbers in binary variables

In the QUBO problems, variables are binaries and we of course want to solve for real numbers in our case. There ar e different ways to encode real numbers in multiple binaries. In our case since the variables are between -1.0 and 1.0 we can use the following encoding : 

$$
r_i = a \sum_n x_n 2^{n} - x_{k+n} 2^{n} 
$$

where $a$ is a normalization constant. THis encoding is created in the `SolutionVector` class that allows to encode/decode real numbers in a series of binaries variables. We use here the `RealUnitQbitEncoding` to obtain real numbers between -1 and 1. The number of qbit controls the precision of the reals we can obtain.

### Polynomial equation

We first write the polynomial equation as follow (https://www.nature.com/articles/s41598-019-46729-0) 

$$
F(X) = 0
$$

with

$$
F_i = P_i^{(0)} + \sum_j P_{ij}^{(1)}x_j + \sum_{jk} P_{ijk}^{(2)}x_j x_k  = 0
$$

To solve the system we minimize the residual sum of square

$$
\chi^2 = [P^{(0)} + P^{(1)} X + P^{(2)} X X.T ]^2
$$


# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
 1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 \\
 c( M - q - p) \rightarrow 0 
$$

with $q$ and $p$ paramerers taking disrete values either 1 or 2

In [14]:
import numpy as np
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 5
    num_variables = 6
    c = 1E-1
    M = 14

    P0 = np.zeros((num_equations,1))
    P0[0] = 2
    P0[1] = 1
    P0[2] = 2
    P0[3] = 0
    P0[4] = c*M

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = -1
    P1[0, 1] =  1

    P1[1, 1] = -1

    P1[2, 2] = -1

    P1[3, 2] =  1 
    P1[3, 3] = -1

    # cost
    P1[4,4] = -c
    P1[4,5] = -c
   

    P2 = np.zeros((num_equations, num_variables, num_variables))


    P3 = np.zeros((num_equations, num_variables, num_variables, num_variables))
    P3[2, 0, 0, 4] = -1
    P3[3, 1, 1, 5] = -1


    return sparse.COO(P0), sparse.COO(P1), sparse.COO(P2), sparse.COO(P3)

matrices = define_matrices()

## 3. Solving the system

We will use here the `SimulatedAnnealingSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.

In [15]:
from qubols.solution_vector import SolutionVector
from qubols.mixed_solution_vector import MixedSolutionVector
from qubols.encodings import PositiveQbitEncoding, RangedEfficientEncoding 

sol_vec0 = SolutionVector(4,2,PositiveQbitEncoding)
sol_vec1 = SolutionVector(2,3, PositiveQbitEncoding)

x0 = sol_vec0.create_polynom_vector()
x1 = sol_vec1.create_polynom_vector()

msv = MixedSolutionVector([sol_vec0, sol_vec1])

# Classical solution 
Enueration of the possible values using Newton Raphson

In [16]:
from quantum_newton_raphson.newton_raphson import newton_raphson
import itertools

values = sol_vec1.encoded_reals[0].get_possible_values()
parameters_list = itertools.product(values, repeat=2)

for parameters in parameters_list:
    initial_point = np.random.rand(4)
    res = newton_raphson(nlfunc, initial_point)
    assert np.allclose(nlfunc(res.solution), 0)
    print(parameters, res.solution)

(0.0, 0.0) [1.5 1.  2.  2. ]
(0.0, 1.0) [1.5 1.  2.  1. ]
(0.0, 2.0) [1.50000000e+00 1.00000000e+00 2.00000000e+00 5.33392774e-14]
(0.0, 3.0) [ 1.5  1.   2.  -1. ]
(0.0, 4.0) [ 1.5  1.   2.  -2. ]
(0.0, 5.0) [ 1.5  1.   2.  -3. ]
(0.0, 6.0) [ 1.5  1.   2.  -4. ]
(0.0, 7.0) [ 1.5  1.   2.  -5. ]
(1.0, 0.0) [ 1.5   1.   -0.25 -0.25]
(1.0, 1.0) [ 1.5   1.   -0.25 -1.25]
(1.0, 2.0) [ 1.5   1.   -0.25 -2.25]
(1.0, 3.0) [ 1.5   1.   -0.25 -3.25]
(1.0, 4.0) [ 1.5   1.   -0.25 -4.25]
(1.0, 5.0) [ 1.5   1.   -0.25 -5.25]
(1.0, 6.0) [ 1.5   1.   -0.25 -6.25]
(1.0, 7.0) [ 1.5   1.   -0.25 -7.25]
(2.0, 0.0) [ 1.5  1.  -2.5 -2.5]
(2.0, 1.0) [ 1.5  1.  -2.5 -3.5]
(2.0, 2.0) [ 1.5  1.  -2.5 -4.5]
(2.0, 3.0) [ 1.5  1.  -2.5 -5.5]
(2.0, 4.0) [ 1.5  1.  -2.5 -6.5]
(2.0, 5.0) [ 1.5  1.  -2.5 -7.5]
(2.0, 6.0) [ 1.5  1.  -2.5 -8.5]
(2.0, 7.0) [ 1.5  1.  -2.5 -9.5]
(3.0, 0.0) [ 1.5   1.   -4.75 -4.75]
(3.0, 1.0) [ 1.5   1.   -4.75 -5.75]
(3.0, 2.0) [ 1.5   1.   -4.75 -6.75]
(3.0, 3.0) [ 1.5   1.   -4.75 -7.

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


In [7]:
from qubols.qubo_poly_mixed_variables import QUBO_POLY_MIXED 
qubo = QUBO_POLY_MIXED(msv)

In [11]:
# create the bqm
bqm = qubo.create_bqm(matrices, strength=1000)

# add constraint
slacks2 = bqm.add_linear_inequality_constraint(qubo.all_expr[3], lagrange_multiplier=1, label="head2", lb=1, ub=2)

# sample
sampleset = qubo.sample_bqm(bqm, num_reads=1000)

# decode
sol, param = qubo.decode_solution(sampleset.lowest())
print(sol, param)

[0.0, 1.0, 2.0, 1.0] [7.0, 1.0]


In [16]:
parameters = param
nlfunc(sol)

array([0., 0., 0., 0.])

In [27]:
import numpy as np
v = np.array([0.25,0.66, 0.89])
a = np.array([[1,0,0],[1,1,0],[1,0,2]])
c = np.linalg.lstsq(a,v)

/tmp/ipykernel_5307/3483934642.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c = np.linalg.lstsq(a,v)


In [28]:
a @ c[0]

array([0.25, 0.66, 0.89])

In [29]:
c

(array([0.25, 0.41, 0.32]),
 array([], dtype=float64),
 3,
 array([2.37607898, 1.41421356, 0.59518794]))

In [34]:
[float(i) for i in np.binary_repr(3,width=4)][::-1]

[1.0, 1.0, 0.0, 0.0]

In [56]:
A

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 2., 0., 0., 0., 0., 0., 0.],
       [1., 1., 2., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 4., 0., 0., 0., 0., 0.],
       [1., 1., 0., 4., 0., 0., 0., 0., 0.]])

In [58]:
values = np.sort(np.random.rand(6))
values

array([0.17863309, 0.26813555, 0.39977383, 0.40088052, 0.44952363,
       0.57442723])

In [63]:
def get_coefs(values, nqbit):
    nvalues = len(values)
    A = np.zeros((nvalues, nqbit + 1))
    c = [1]+[2**i for i in range(nqbit)]
    for idx in range(nvalues):
        row = [1] + [float(i) for i in np.binary_repr(idx, width=nqbit)][::-1]
        A[idx, :] = row
    A = A*c
    coefs, _, _, _ = np.linalg.lstsq(A,values)
    return A, coefs

In [65]:
c = get_coefs(np.sort(np.random.rand(6)),8)


/tmp/ipykernel_5307/3504057229.py:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, _, _, _ = np.linalg.lstsq(A,values)
